In [1]:
###import comstockpostproc-standard things, and then don't use most of them

import os
from textwrap import indent

import boto3
import logging
import numpy as np
import pandas as pd

from comstockpostproc.resstock_naming_mixin_LARGEE import ResStockNamingMixin
from comstockpostproc.units_mixin import UnitsMixin
from comstockpostproc.s3_utilities_mixin import S3UtilitiesMixin
from comstockpostproc import resstock_LARGEE

logger = logging.getLogger(__name__)

In [2]:
### specify (most) inputs
resstock_results_folder = "C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Data/2024.2/AMY2018"
col_plan_folder = 'C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Baseline (4.2)'
output_folder = "C:/Users/epresent/NREL/BuildStock Analysis User Engagement-C2C Delaware - Documents/10_Analysis/Baseline (4.2)"

results_file_name = "PA_baseline_metadata_and_annual_results.csv"
col_plan_name = 'Column Plan.csv'
output_file_name = 'ResStock_long_v4_PA.csv'

downselect_row_fields = ["in.county_name"]
values_to_keep = [["Montgomery County", "Bucks County", "Chester County", "Delaware County"]]

In [3]:
###load results from already-downloaded OEDI file
results_file_path = os.path.join(resstock_results_folder, results_file_name)
data = pd.read_csv(results_file_path, engine="pyarrow")

In [4]:
### Downselect rows
for field, values in zip(downselect_row_fields, values_to_keep):
    data = data.loc[data[field].isin(values)]

In [ ]:
### Add general fields

In [4]:
###make plan for each column

# load column plan
plan_file_path = os.path.join(col_plan_folder, col_plan_name)
col_plan = pd.read_csv(plan_file_path, engine = "pyarrow")
#create lists of columns
cols_to_remove = col_plan.loc[col_plan['plan']=='remove', 'column'].tolist()
cols_wide = col_plan.loc[col_plan['plan']=='keep', 'column'].tolist()
cols_to_pivot = col_plan.loc[col_plan['plan']=='pivot', 'column'].tolist()

In [5]:
#remove unneccsary columns
data.drop(data[cols_to_remove], axis = 1, inplace = True)

In [6]:
###add project-specific long fields

##project-specific utility bills - inputs
#Electricity
fixed_elec_cost_monthly = 10.56
var_elec_cost_per_kwh = 0.17404 #cf 0.137/kwh

#Natural Gas
fixed_ng_cost_monthly = 16.25
var_ng_cost_per_ccf = 1.495

#Fuel Oil
var_fo_cost_per_gal = 2.851

#Propane
var_propane_cost_per_gal = 3.199


##project-specific utility bills - unit conversions
gal_fuel_oil_to_mbtu = 139/1000
gal_propane_to_mbtu = 91.6 / 1000
mbtu_to_kwh = 293.0710701722222
dol_per_ccf_to_dol_per_therm = 1/1.038 #$ per Ccf divided by 1.038 equals $ per therm https://www.eia.gov/tools/faqs/faq.php?id=45&t=8
therm_to_kwh = 29.307107017222222

var_ng_cost_per_kwh = var_ng_cost_per_ccf * (dol_per_ccf_to_dol_per_therm) * (1/therm_to_kwh)
var_fo_cost_per_kwh = var_fo_cost_per_gal * (1/gal_fuel_oil_to_mbtu) * (1/mbtu_to_kwh)
var_propane_cost_per_kwh = var_propane_cost_per_gal * (1/gal_propane_to_mbtu) * (1/mbtu_to_kwh)

#print(var_ng_cost_per_kwh) #0.011423113848862812, cf 0.0339307/kwh
#print(var_fo_cost_per_kwh) #0.06998572515142275, cf 0.0704125/kwh
#print(var_propane_cost_per_kwh) #0.11916420397790706. cf 0.101456/kWh

##project-specific utility bills - calculations and add to dataset
data["out.bills_local.electricity.total.usd"] = (fixed_elec_cost_monthly*12) + (var_elec_cost_per_kwh * data["out.electricity.total.energy_consumption.kwh"])
data["out.bills_local.natural_gas.total.usd"] = (fixed_ng_cost_monthly*12) + (var_ng_cost_per_kwh * data["out.natural_gas.total.energy_consumption.kwh"])
data["out.bills_local.fuel_oil.total.usd"] = (var_fo_cost_per_kwh * data["out.fuel_oil.total.energy_consumption.kwh"])
data["out.bills_local.propane.total.usd"] = (var_propane_cost_per_kwh * data["out.propane.total.energy_consumption.kwh"])
data["out.bills_local.all_fuels.total.usd"] = data["out.bills_local.electricity.total.usd"] + data["out.bills_local.natural_gas.total.usd"] + data["out.bills_local.fuel_oil.total.usd"] + data["out.bills_local.propane.total.usd"]

##add new fields to column plan
cols_to_pivot = cols_to_pivot + ["out.bills_local.electricity.total.usd", "out.bills_local.natural_gas.total.usd", "out.bills_local.fuel_oil.total.usd", "out.bills_local.propane.total.usd",
                                 "out.bills_local.all_fuels.total.usd"]
plan_for_new_cols = [
    ["out.bills_local.electricity.total.usd", "out.x", "keep", "Utility Bills", "Electricity", "Electricity Total", "Total"],
    ["out.bills_local.natural_gas.total.usd", "out.x", "keep", "Utility Bills", "Natural Gas", "Natural Gas Total", "Total"],
    ["out.bills_local.fuel_oil.total.usd", "out.x", "keep", "Utility Bills", "Fuel Oil", "Fuel Oil Total", "Total"],
    ["out.bills_local.propane.total.usd", "out.x", "keep", "Utility Bills", "Propane", "Propane Total", "Total"],
    ["out.bills_local.all_fuels.total.usd", "out.x", "keep", "Utility Bills", "Energy", "Total", "Total"]
]
plan_for_new_cols_df = pd.DataFrame(plan_for_new_cols, columns = 
                                    ['column', 'col_type', 'plan', 'Result Type', 'Fuel', 'End Use', 'End Use Category'])
col_plan = pd.concat([col_plan, plan_for_new_cols_df], axis = 0)

In [7]:
###pivot the data
data_long = pd.melt(
    data,
    id_vars = cols_wide,
    var_name = 'Output',
    value_name = 'Value'
)

In [8]:
###Develop output categorization
#split original column name
data_long[['out', 'Result', 'End Use', 'Type', 'Units']] = data_long['Output'].str.split('.', expand = True)
#only keep the units
data_long.drop(data_long[['out', 'Result', 'End Use', 'Type']], axis = 1, inplace = True)
#use mappings to get the categorizations
out_cats = col_plan.drop(col_plan[["col_type", "plan"]], axis = 1, inplace = False)
data_long = data_long.merge(out_cats, left_on = 'Output', right_on = "column", how = 'left')



In [ ]:
### Add long file format fields


In [9]:
### Add project-specific long fields
## total utility bill, total emissions
data_long = data_long.merge(data[["bldg_id", "out.bills_local.all_fuels.total.usd", "out.emissions.all_fuels.lrmer_mid_case_15.co2e_kg"]], on="bldg_id", how="left")
data_long.rename(columns = {"out.emissions.all_fuels.lrmer_mid_case_15.co2e_kg":"Emissions", "out.bills_local.all_fuels.total.usd":"Utility Bill Total"}, inplace=True)

In [10]:
### Save long file

data_long.to_csv(os.path.join(output_folder, output_file_name))